# 02 – Islet Detection & Radial Analysis

This notebook performs:
1. **Islet detection** – Identify islet-like endocrine structures vs. single cells
2. **Merge fragmented islets** – Combine nearby endocrine regions
3. **CD3 T-cell mapping** – Count immune cells within/around islets
4. **Radial distance analysis** – Measure cell distributions at stepwise distances from islet centroids
5. **Comprehensive visualizations** – Full-image and close-up views with overlays

Handles irregular islet shapes common in diseased pancreas tissue.

In [ ]:
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

os.environ.setdefault('OPENCV_IO_MAX_IMAGE_PIXELS', str(2**63 - 1))

import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.colors import ListedColormap
from skimage.color import label2rgb
from skimage.segmentation import find_boundaries
from scipy import ndimage

from isletscope.islet_detection import IsletDetector
from isletscope.radial_analysis import RadialAnalyzer

print('Imports complete')

## Configuration

In [ ]:
# ===== Input =====
output_dir = Path('../outputs')
output_dir.mkdir(exist_ok=True)

# Load results from notebook 01
img_normalized_path = output_dir / 'img_normalized.npy'
cell_mask_path = output_dir / 'cell_mask.npy'
cell_labels_path = output_dir / 'cell_labels.npy'

# Optional: specific marker masks
insulin_mask_path = output_dir / 'marker_insulin.npy'
glucagon_mask_path = output_dir / 'marker_glucagon.npy'
cd3_mask_path = output_dir / 'marker_CD3.npy'

# ===== Islet Detection =====
min_islet_area = 500  # Minimum pixels for islet vs. single cell
min_cell_count = 10  # Minimum cells per islet
merge_distance = 8  # Merge endocrine regions within this distance
min_density = 0.2  # Minimum cell density to be considered islet

# ===== Radial Analysis =====
bin_size = 20  # Radial bin width in pixels
distance_strategy = 'boundary'  # 'centroid' or 'boundary'

# ===== Visualization =====
closeup_regions = [
    (0.2, 0.4, 0.3, 0.5),  # Region 1 (contains islet)
    (0.5, 0.7, 0.6, 0.8),  # Region 2
]

print('Configuration set')

## Load Data

In [ ]:
# Load required arrays
img_normalized = np.load(img_normalized_path)
cell_mask = np.load(cell_mask_path)
cell_labels = np.load(cell_labels_path)

print(f'Image shape: {img_normalized.shape}')
print(f'Cell labels: {cell_labels.max()} cells')

# Load markers if available
insulin_mask = np.load(insulin_mask_path) if insulin_mask_path.exists() else None
glucagon_mask = np.load(glucagon_mask_path) if glucagon_mask_path.exists() else None
cd3_mask = np.load(cd3_mask_path) if cd3_mask_path.exists() else None

# Create endocrine mask (insulin OR glucagon positive cells)
if insulin_mask is not None and glucagon_mask is not None:
    endocrine_mask = (insulin_mask | glucagon_mask).astype(np.uint8)
    print(f'Endocrine cells: {endocrine_mask.sum()} (insulin: {insulin_mask.sum()}, glucagon: {glucagon_mask.sum()})')
else:
    # Fallback: assume all cells are potential endocrine
    endocrine_mask = (cell_mask > 0).astype(np.uint8)
    print('No marker data; using all cells as endocrine candidates')

if cd3_mask is not None:
    print(f'CD3+ T-cells: {cd3_mask.sum()}')

def get_closeup_coords(img_shape, region_frac):
    h, w = img_shape[:2]
    y1, y2, x1, x2 = region_frac
    return (int(y1*h), int(y2*h), int(x1*w), int(x2*w))

## Islet Detection

Distinguish islets (clustered endocrine cells) from single endocrine cells.

In [ ]:
print('Detecting islets...')
detector = IsletDetector(
    min_islet_area=min_islet_area,
    min_cell_count=min_cell_count,
    merge_distance=merge_distance,
    min_density=min_density
)

islet_result = detector.detect(
    endocrine_mask=endocrine_mask,
    cell_labels=cell_labels,
    cd3_mask=cd3_mask
)

islet_labels = islet_result['islet_labels']
single_mask = islet_result['single_mask']
islet_metrics = islet_result['metrics']

n_islets = islet_labels.max()
n_singles = single_mask.sum()

print(f'\nDetected {n_islets} islets')
print(f'Single endocrine cells: {n_singles} pixels')
print(f'\nIslet metrics (first 5):')
for i, m in enumerate(islet_metrics[:5], 1):
    print(f"  Islet {i}: area={m['area']} px, cells={m['cell_count']}, density={m['density']:.2f}")
    if 'cd3_count' in m:
        print(f"           CD3={m['cd3_count']}")

In [ ]:
# Full image visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 16))

# Original image
axes[0, 0].imshow(cv2.cvtColor(img_normalized, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Normalized Image', fontsize=14, fontweight='bold')
axes[0, 0].axis('off')

# Endocrine mask
axes[0, 1].imshow(endocrine_mask, cmap='Reds')
axes[0, 1].set_title(f'Endocrine Cells ({endocrine_mask.sum()} px)', fontsize=14, fontweight='bold')
axes[0, 1].axis('off')

# Islet labels
islet_colored = label2rgb(islet_labels, bg_label=0, bg_color=(0, 0, 0))
axes[1, 0].imshow(islet_colored)
axes[1, 0].set_title(f'Detected Islets ({n_islets})', fontsize=14, fontweight='bold')
axes[1, 0].axis('off')

# Overlay: islets + singles + CD3
overlay = cv2.cvtColor(img_normalized, cv2.COLOR_BGR2RGB).copy()
# Islets in green
overlay[islet_labels > 0] = overlay[islet_labels > 0] * 0.5 + np.array([0, 255, 0]) * 0.5
# Singles in red
overlay[single_mask > 0] = overlay[single_mask > 0] * 0.5 + np.array([255, 0, 0]) * 0.5
# CD3 in blue
if cd3_mask is not None:
    overlay[cd3_mask > 0] = overlay[cd3_mask > 0] * 0.5 + np.array([0, 100, 255]) * 0.5

axes[1, 1].imshow(overlay)
title = 'Overlay: Islets (green), Singles (red)'
if cd3_mask is not None:
    title += ', CD3 (blue)'
axes[1, 1].set_title(title, fontsize=14, fontweight='bold')
axes[1, 1].axis('off')

# Mark closeup regions
for i, region_frac in enumerate(closeup_regions):
    y1, y2, x1, x2 = get_closeup_coords(img_normalized.shape, region_frac)
    rect = Rectangle((x1, y1), x2-x1, y2-y1, linewidth=2, edgecolor='yellow', facecolor='none')
    axes[1, 1].add_patch(rect)
    axes[1, 1].text(x1, y1-10, f'Region {i+1}', color='yellow', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig(output_dir / '03_islet_detection_full.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Closeup views
n_regions = len(closeup_regions)
fig, axes = plt.subplots(n_regions, 4, figsize=(20, 5*n_regions))
if n_regions == 1:
    axes = axes.reshape(1, -1)

for i, region_frac in enumerate(closeup_regions):
    y1, y2, x1, x2 = get_closeup_coords(img_normalized.shape, region_frac)
    
    img_crop = img_normalized[y1:y2, x1:x2]
    endo_crop = endocrine_mask[y1:y2, x1:x2]
    islet_crop = islet_labels[y1:y2, x1:x2]
    single_crop = single_mask[y1:y2, x1:x2]
    
    # Image
    axes[i, 0].imshow(cv2.cvtColor(img_crop, cv2.COLOR_BGR2RGB))
    axes[i, 0].set_title(f'Region {i+1} - Image', fontsize=12, fontweight='bold')
    axes[i, 0].axis('off')
    
    # Endocrine mask
    axes[i, 1].imshow(endo_crop, cmap='Reds')
    axes[i, 1].set_title(f'Region {i+1} - Endocrine Cells', fontsize=12, fontweight='bold')
    axes[i, 1].axis('off')
    
    # Islet labels
    islet_colored_crop = label2rgb(islet_crop, bg_label=0)
    axes[i, 2].imshow(islet_colored_crop)
    n_islets_crop = islet_crop.max()
    axes[i, 2].set_title(f'Region {i+1} - Islets ({n_islets_crop})', fontsize=12, fontweight='bold')
    axes[i, 2].axis('off')
    
    # Overlay with boundaries
    overlay_crop = cv2.cvtColor(img_crop, cv2.COLOR_BGR2RGB).copy()
    # Draw islet boundaries in thick yellow
    if islet_crop.max() > 0:
        boundaries = find_boundaries(islet_crop, mode='thick')
        overlay_crop[boundaries] = [255, 255, 0]
    # Highlight singles in red
    overlay_crop[single_crop > 0] = [255, 0, 0]
    
    axes[i, 3].imshow(overlay_crop)
    axes[i, 3].set_title(f'Region {i+1} - Boundaries', fontsize=12, fontweight='bold')
    axes[i, 3].axis('off')

plt.tight_layout()
plt.savefig(output_dir / '03_islet_detection_closeup.png', dpi=150, bbox_inches='tight')
plt.show()

## Radial Analysis

Compute stepwise radial distances from islet centroids/boundaries and bin cell counts.

In [ ]:
print(f'Running radial analysis (strategy: {distance_strategy}, bin size: {bin_size} px)...')
analyzer = RadialAnalyzer(bin_size=bin_size, distance_strategy=distance_strategy)

# Prepare cell masks for binning
cell_masks = {'all_cells': (cell_mask > 0).astype(np.uint8)}
if cd3_mask is not None:
    cell_masks['CD3'] = cd3_mask
if insulin_mask is not None:
    cell_masks['insulin'] = insulin_mask
if glucagon_mask is not None:
    cell_masks['glucagon'] = glucagon_mask

radial_results = analyzer.analyze_islets(islet_labels, cell_masks=cell_masks)

print(f'\nRadial analysis complete for {len(radial_results)} islets')
print(f'Negative bins = inside islet, positive bins = outside islet')

In [ ]:
# Radial heatmap for first islet
if radial_results:
    islet_id = list(radial_results.keys())[0]
    result = radial_results[islet_id]
    distance_map = result['distance_map']
    bins_data = result['bins']
    
    # Find islet bounding box
    islet_mask = (islet_labels == islet_id)
    coords = np.argwhere(islet_mask)
    y_min, x_min = coords.min(axis=0)
    y_max, x_max = coords.max(axis=0)
    
    # Add padding
    pad = 100
    y1 = max(0, y_min - pad)
    y2 = min(distance_map.shape[0], y_max + pad)
    x1 = max(0, x_min - pad)
    x2 = min(distance_map.shape[1], x_max + pad)
    
    dist_crop = distance_map[y1:y2, x1:x2]
    img_crop = img_normalized[y1:y2, x1:x2]
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # Original
    axes[0].imshow(cv2.cvtColor(img_crop, cv2.COLOR_BGR2RGB))
    axes[0].set_title(f'Islet {islet_id} - Image', fontsize=12, fontweight='bold')
    axes[0].axis('off')
    
    # Distance map
    im = axes[1].imshow(dist_crop, cmap='RdBu_r', vmin=-200, vmax=200)
    axes[1].set_title(f'Islet {islet_id} - Radial Distance Map', fontsize=12, fontweight='bold')
    axes[1].axis('off')
    plt.colorbar(im, ax=axes[1], label='Distance (px, negative=inside)')
    
    # Overlay with contours
    overlay_crop = cv2.cvtColor(img_crop, cv2.COLOR_BGR2RGB).copy()
    islet_crop = islet_labels[y1:y2, x1:x2]
    boundaries = find_boundaries(islet_crop, mode='thick')
    overlay_crop[boundaries] = [255, 255, 0]
    axes[2].imshow(overlay_crop)
    axes[2].set_title(f'Islet {islet_id} - Boundary', fontsize=12, fontweight='bold')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.savefig(output_dir / f'04_radial_distance_islet{islet_id}.png', dpi=150, bbox_inches='tight')
    plt.show()

In [ ]:
# Plot radial distributions for first few islets
n_plot = min(3, len(radial_results))
fig, axes = plt.subplots(1, n_plot, figsize=(6*n_plot, 5))
if n_plot == 1:
    axes = [axes]

for idx, (islet_id, result) in enumerate(list(radial_results.items())[:n_plot]):
    bins_data = result['bins']
    
    # Plot each cell type
    for cell_type, bin_dict in bins_data.items():
        bin_centers = sorted(bin_dict.keys())
        counts = [bin_dict[b] for b in bin_centers]
        axes[idx].plot(bin_centers, counts, marker='o', label=cell_type)
    
    axes[idx].axvline(0, color='red', linestyle='--', linewidth=1, alpha=0.5, label='Boundary')
    axes[idx].set_xlabel('Radial Distance (px)', fontsize=11)
    axes[idx].set_ylabel('Cell Count', fontsize=11)
    axes[idx].set_title(f'Islet {islet_id} - Radial Distribution', fontsize=12, fontweight='bold')
    axes[idx].legend()
    axes[idx].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / '04_radial_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

## Save Results

In [ ]:
# Save outputs
np.save(output_dir / 'islet_labels.npy', islet_labels)
np.save(output_dir / 'single_mask.npy', single_mask)
np.save(output_dir / 'endocrine_mask.npy', endocrine_mask)

# Save radial results
import pickle
with open(output_dir / 'radial_results.pkl', 'wb') as f:
    pickle.dump(radial_results, f)

# Save islet metrics as CSV
import pandas as pd
metrics_df = pd.DataFrame(islet_metrics)
metrics_df.to_csv(output_dir / 'islet_metrics.csv', index=False)

print(f'Results saved to {output_dir}')
print(f'\nIslet metrics saved to: {output_dir / "islet_metrics.csv"}')
print(f'Radial results saved to: {output_dir / "radial_results.pkl"}')
print('\nReady for notebook 03: Tissue Classification & 3D Inference')